In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [4]:
import math
import random
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.states.simple_rotation_generators import get_binary_x_rotation_provider 
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r, get_g_parameters_provider
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [5]:
generator_layers = 3
discriminator_layers = 5
data_bus_size = 3

In [6]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [7]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [8]:
real, real_symbols = build_x_rotation_state(qubits=data_qubits)

In [9]:
pure_gen = gen.copy()
gen.append([disc])

In [10]:
pure_real = real.copy()
real.append([disc])

In [11]:
print("REAL GROUND STATE")
print(pure_real)

REAL GROUND STATE
(0, 2): ───Rx(r0)───

(0, 3): ───Rx(r1)───

(0, 4): ───Rx(r2)───


In [12]:
print("GENERATOR")
print(pure_gen)

GENERATOR
(0, 2): ───Rx(l)───Rx(g0)───Rz(g4)───ZZ───────────────Rx(g11)───Rz(g15)───ZZ────────────────Rx(g22)───Rz(g26)───ZZ────────────────
                                     │                                    │                                     │
(0, 3): ───Rx(l)───Rx(g1)───Rz(g5)───ZZ^g8───ZZ───────Rx(g12)───Rz(g16)───ZZ^g19───ZZ───────Rx(g23)───Rz(g27)───ZZ^g30───ZZ───────
                                             │                                     │                                     │
(0, 4): ───Rx(l)───Rx(g2)───Rz(g6)───ZZ──────ZZ^g10───Rx(g13)───Rz(g17)───ZZ───────ZZ^g21───Rx(g24)───Rz(g28)───ZZ───────ZZ^g32───
                                     │                                    │                                     │
(0, 5): ───Rx(l)───Rx(g3)───Rz(g7)───ZZ^g9────────────Rx(g14)───Rz(g18)───ZZ^g20────────────Rx(g25)───Rz(g29)───ZZ^g31────────────


In [13]:
print("DISCRIMINATOR")
print(disc)

DISCRIMINATOR
(0, 0): ───Rx(l)───Rx(d0)───Rz(d5)───ZZ────────────────Rx(d14)───Rz(d19)───ZZ────────────────Rx(d28)───Rz(d33)───ZZ────────────────Rx(d42)───Rz(d47)───ZZ────────────────Rx(d56)───Rz(d61)───ZZ────────────────
                                     │                                     │                                     │                                     │                                     │
(0, 1): ───Rx(l)───Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────Rx(d29)───Rz(d34)───ZZ^d38───ZZ───────Rx(d43)───Rz(d48)───ZZ^d52───ZZ───────Rx(d57)───Rz(d62)───ZZ^d66───ZZ───────
                                              │                                     │                                     │                                     │                                     │
(0, 2): ───Rx(l)───Rx(d2)───Rz(d7)───ZZ───────ZZ^d12───Rx(d16)───Rz(d21)───ZZ───────ZZ^d26───Rx(d30)───Rz(d35)───ZZ───────ZZ^d40───Rx(d44)───Rz(d49)───ZZ───────ZZ^d54───Rx(d58)───

In [14]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [15]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [16]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [17]:
g_provider = lambda: random.choice([0, 1, 2])
x_rotations = get_binary_x_rotation_provider({0: '100', 1: '011', 2: '101'})

In [18]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  real_values_provider = x_rotations,
                  use_analytical_expectation=True)

In [ ]:
epochs = 101
disc_iteration = 100
gen_iteration = 10
print_interval_epoch = 10
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      print_interval_epoch=print_interval_epoch)

In [19]:
def get_gen_for_g(g, gen_weights):
    rad = map_to_radians(g)
    return np.append(gen_weights, rad)

def get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, gen_weights, g, size, real_provider):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights), list(range(size)))
    real = real_evaluator.get_state_from_params(real_provider(g), list(range(size)))
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, size):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights), list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, ground, cirq.fidelity(generated, ground)

def compare_generated_for_g(gen_evaluator, g1, g2, gen_weights, size):
     generated1 = gen_evaluator.get_state_from_params(get_gen_for_g(g1, gen_weights), list(range(size)))
     generated2 = gen_evaluator.get_state_from_params(get_gen_for_g(g2, gen_weights), list(range(size)))
     return generated1, generated2, cirq.fidelity(generated1, generated2)

In [20]:
trained_disc_weights = tf.Variable(np.array([ 1.25888796e+01,  1.10409822e+01,  1.27487049e+01,  1.32927475e+01,
       -3.20522385e+01,  2.98508596e+00, -7.54223883e-01,  8.97036648e+00,
        8.98472309e+00, -2.77423954e+00,  8.90891266e+00,  5.72837019e+00,
        6.03105211e+00, -4.64482594e+00, -1.10843427e-01,  7.78598115e-02,
        3.00343895e+00,  3.83781940e-01,  6.08641243e+00,  8.64131927e+00,
       -2.17593918e+01,  1.45857897e+01,  2.36893883e+01,  8.75363445e+00,
       -2.42768993e+01,  1.27688437e+01,  2.53628349e+00,  1.39768391e+01,
        1.40961084e+01, -2.04474068e+01,  4.71392822e+00,  1.66926212e+01,
        1.88311214e+01,  9.17525005e+00,  1.16109962e+01,  1.08004580e+01,
        1.10795708e+01,  5.81477690e+00, -5.50215101e+00,  2.22007637e+01,
        2.25015125e+01,  3.21826210e+01, -1.80058708e+01, -7.85126591e+00,
       -7.77073908e+00,  1.40237570e+01,  3.14071465e+01, -1.07477732e+01,
        1.55852342e+00,  2.63786411e+01,  1.31890945e+01,  7.14759350e+00,
        5.46145630e+00, -1.79730053e+01,  7.40563774e+00,  1.06135674e+01,
       -1.55095673e+00, -2.66580944e+01,  6.51995277e+00, -2.37151980e+00,
        3.51896515e+01,  1.50027342e+01,  1.80733763e-02, -1.75052185e+01,
        9.60706902e+00, -6.06413984e+00,  5.35433817e+00, -7.88707399e+00,
        1.27363043e+01,  1.02446747e+01]), dtype=tf.float32)

trained_gen_weights = np.array([ -1.6883899 ,  12.534816  ,   0.3735907 ,  -9.315787  ,
        -6.007224  , -12.969288  ,   8.700528  ,  -5.1759815 ,
         4.310844  ,  -3.130318  ,   6.798724  ,   4.905171  ,
        -0.2890863 ,  -1.222973  ,  -0.07461139,  -2.5728188 ,
         1.7303827 ,  -1.5344989 ,   2.6312382 ,  -1.8737797 ,
        -1.0022153 ,   4.3546224 ,   3.5610907 ,  -4.2001524 ,
        10.083789  , -16.943022  ,   8.448887  ,   7.81465   ,
         2.9656339 ,   0.04719266,  14.356344  ,   0.31659052,
         4.6315327 ])

gen_symbols = gs + (ls,)
# real_symbols = ('r0', 'r1', 'r2')

In [21]:
gen_evaluator = CircuitEvaluator(pure_gen, gen_symbols)
real_evaluator = CircuitEvaluator(pure_real, real_symbols)

In [22]:
g = 0
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights, g, data_bus_size, get_g_parameters_provider())

Max probability state after tracing has probability: 0.8652666211128235
Max probability state after tracing has probability: 1.0


(array([-0.59494203+0.j        ,  0.11035448-0.21411993j,
        -0.11897308-0.20157549j, -0.3484044 +0.11465914j,
         0.06583245+0.3389708j , -0.05640569+0.01146295j,
         0.35240868+0.05967778j, -0.02327937-0.38451982j], dtype=complex64),
 array([ 0.       -0.j       , -0.477583 +0.5214543j,
         0.       -0.j       ,  0.5214543+0.477583j ,
         0.       -0.j       ,  0.       -0.j       ,
         0.       -0.j       ,  0.       -0.j       ], dtype=complex64),
 0.15822610790345326)

In [23]:
g = -1
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights, g, data_bus_size, get_g_parameters_provider())

Max probability state after tracing has probability: 0.8131296038627625
Max probability state after tracing has probability: 0.9999998807907104


(array([-0.34420297+0.j        ,  0.0109985 -0.12667517j,
         0.505712  -0.05309889j, -0.2982401 -0.07065247j,
         0.22529078-0.2594643j ,  0.09571937+0.04974068j,
         0.5242721 -0.07012914j, -0.3077103 -0.09313883j], dtype=complex64),
 array([ 2.5000000e-01+0.0000000e+00j, -1.1102230e-16-2.5000000e-01j,
         2.0376244e-17-6.0355341e-01j, -6.0355341e-01+6.4979843e-18j,
         2.6728770e-19-1.0355339e-01j, -1.0355339e-01-3.6265658e-18j,
        -2.5000000e-01+4.4419206e-18j,  4.4419206e-18+2.5000000e-01j],
       dtype=complex64),
 0.21501619424179808)

In [ ]:
compare_generated_for_g(gen_evaluator, -1, 0, trained_gen_weights, data_bus_size)

In [ ]:
g = 0
get_states_and_fidelty_for_ground(gen_evaluator, g, trained_gen_weights, data_bus_size)